# sklearn IRIS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [2]:
import bentoml
NAME = 'sklearn_iris'

# load data & train model

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.65 0.   1.21 1.   0.   1.   2.   1.   0.   2.
 0.   1.88 2.   2.   0.   0.   1.   2.   1.   1.36 1.49 1.88 1.   1.
 2.   2.  ]
  mse: 0.087503


In [9]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):
    @bentoml.api(DataframeHandler)
    def predict(self, inputs):
        outputs = self.artifacts.model.predict(inputs.to_numpy())
        return outputs

Overwriting sklearn_iris.py


In [12]:
from sklearn_iris import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-03-19 16:37:17,373] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-03-19 16:37:27,241] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.5.2+137.gcbdaab7
creating BentoML-0.5.2+137.gcbdaab7/BentoML.egg-info
creating BentoML-0.5.2+137.gcbdaab7/bentoml
creating BentoML-0.5.2+137.gcbdaab7/bentoml/artifact
creating BentoML-0.5.2+137.gcbdaab7/bentoml/bundler
creating BentoML-0.5.2+137.gcbdaab7/bentoml/cli
creating BentoML-0.5.2+137.gcbdaab7/bentoml/clipper
creating BentoML-0.5.2+137.gcbdaab7/bentoml/configuration
creating BentoML-0.5.2+137.gcbdaab7/bentoml/configuration/__pycache__
creating BentoML-0.5.2+137.gcbdaab7/bentoml/deployment
creating BentoML-0.5.2+137.gcbdaab7/bentoml/deployment/aws_lambda
creating BentoML-0.5.2+137.gcbdaab7/bentoml/deployment/sagemaker
creating BentoML-0.5.2+137.gcbdaab7/bentoml/handlers
creating BentoML-0.5.2+137.gcbdaab7/bentoml/marshal
creating BentoML-0.5.2+137.gcbdaab7/bentoml/migrations
creating BentoML-0.5.2+137.gcbdaab7/bentoml/migrations/__pycache__
creating BentoML-0.5.2+137.gcbdaab7/bentoml/migrations

In [17]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)

36041


In [23]:
print(f"bentoml serve-gunicorn {saved_path} --port {PORT} --workers 1 --enable-microbatch")
!bentoml serve-gunicorn {saved_path} --port {PORT} --workers 1 --enable-microbatch

bentoml serve-gunicorn /home/bentoml/bentoml/repository/BentoSvc/20200319163717_088228 --port 36041 --workers 1 --enable-microbatch
[2020-03-19 16:47:07,627] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-03-19 16:47:07,632] INFO - Running micro batch service on :36041
^C
Process Process-1:

Aborted!


# Build & Run Bento Service in Docker

In [ ]:
!cd {saved_path}
IMG_NAME = saved_path.split('/')[-1].lower()
!docker build -t {IMG_NAME} \
    --build-arg PIP_TRUSTED_HOST=192.168.138.2 \  # set your prefer PYPI mirror
    --build-arg PIP_INDEX_URL=http://192.168.138.2/simple \
    {saved_path}
!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {IMG_NAME}:latest

# Test with requests

In [24]:
import json
import requests
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
raw_data = X_test
data = pd.DataFrame(raw_data).to_json()

json_response = requests.post(f'http://127.0.0.1:{PORT}/predict',
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

<Response [200]>
[2.0, 1.0, 0.0, 1.0, 1.65, 0.0, 1.21, 1.0, 0.0, 1.0, 2.0, 1.0, 0.0, 2.0, 0.0, 1.88, 2.0, 2.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.36, 1.49, 1.88, 1.0, 1.0, 2.0, 2.0]


# Benchmark with locust

In [25]:
%%writefile benchmark_{NAME}.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import pandas as pd
import json


@lru_cache(maxsize=1)
def data_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    headers = {"content-type": "application/json"}

    def _gen_data():
        raw_data = X_test
        data = pd.DataFrame(raw_data).to_json()
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @task
    def index(self):
        headers, data = data_producer()()
        self.client.post("/predict", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(1)

Writing benchmark_sklearn_iris.py


In [27]:
!locust -f benchmark_{NAME}.py -H http://127.0.0.1:{PORT}

[2020-03-19 16:47:52,290] beta/INFO/locust.main: Starting web monitor at http://*:8089
[2020-03-19 16:47:52,290] beta/INFO/locust.main: Starting Locust 0.14.4
[2020-03-19 16:48:06,109] beta/INFO/locust.runners: Hatching and swarming 950 users at the rate 100 users/s (0 users already running)...
[2020-03-19 16:48:17,325] beta/WARNING/root: Loadgen CPU usage above 90%! This may constrain your throughput and may even give inconsistent response time measurements! See https://docs.locust.io/en/stable/running-locust-distributed.html for how to distribute the load over multiple CPU cores or machines
[2020-03-19 16:48:22,522] beta/INFO/locust.runners: All locusts hatched: WebsiteUser: 950 (0 already running)
^C
[2020-03-19 17:04:38,000] beta/ERROR/stderr: KeyboardInterrupt
[2020-03-19 17:04:38,001] beta/ERROR/stderr: 2020-03-19T09:04:37Z
[2020-03-19 17:04:38,001] beta/ERROR/stderr: 
[2020-03-19 17:04:38,001] beta/INFO/locust.main: Shutting down (exit code 0), bye.
[2020-03-19 17:04:38,001] bet